In [1]:
import pandas as pd
import numpy as np
from datetime import *
from glob import glob

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

2.4.0


In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score

In [4]:
import pylab as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

## Get data

In [5]:
with open('pipeline_data/steps.dat', 'r') as handle:
    steps = handle.readline().strip().split()

In [6]:
csv_fname = glob('pipeline_data/data_????-??-??_glm.csv')[0]
df = pd.read_csv(csv_fname, dtype={'fips': str}).set_index('fips')
df.head()

case0  case1  case2  case3  case4  case5  case6  case7  case8  case9  \
fips                                                                          
01001   12.0    7.0    6.0   11.0   11.0   16.0   45.0   48.0   60.0   48.0   
01003   34.0   36.0   40.0   45.0   30.0   34.0   42.0   19.0   12.0   29.0   
01005    3.0    7.0    8.0   16.0    9.0   14.0   24.0   27.0   47.0   41.0   
01007    4.0    9.0   13.0    8.0    7.0    4.0    7.0    7.0   15.0    5.0   
01009   12.0    3.0    9.0   12.0    6.0    5.0    2.0    3.0   13.0   11.0   

       ...  death33_glm  death34_glm  death35_glm  death36_glm  death37_glm  \
fips   ...                                                                    
01001  ...         3.32         2.95         4.44         4.85         5.00   
01003  ...         5.28         5.25         7.80         8.73         9.03   
01005  ...         0.86         0.66         0.98         1.02         1.21   
01007  ...         1.33         1.13         1.67         1.77         1.91   
01009  ...         1.22         1.17         1.64         1.77         1.83   

       death38_glm  death39_glm  death40_glm  death41_glm  total_death_glm  
fips                                                                        
01001         4.48         4.88         6.20         6.33            93.11  
01003         7.65         8.82        11.07        11.58           134.33  
01005         1.38         1.43         2.22         2.62            56.28  
01007         1.74         1.90         2.49         2.54            37.07  
01009         1.50         1.73         2.32         2.42            21.42  

[5 rows x 182 columns]

## Functions

In [7]:
class tfRegr:
    
    def __init__(self, 
                 epoch=100, 
                 verbose=False, 
                 validation_split=0.2, 
                 learning_rate=.1):
        
        self.normalizer = preprocessing.Normalization()
        self.ep = epoch
        self.vb = verbose
        self.vs = validation_split
        self.lr = learning_rate
    
    def fit(self, X, y):
        self.normalizer.adapt(X)

        self.model = tf.keras.Sequential([
            self.normalizer,
            layers.Dense(units=1)
        ])

        self.model.compile(
            optimizer=tf.optimizers.Adam(learning_rate=self.lr),
            loss='mean_absolute_error'
        )

        self.history = self.model.fit(
            X, y,
            epochs=self.ep,
            verbose=self.vb, # logging
            validation_split=self.vs # Calculate validation results on a fraction of the training data
        )
        return self.model

In [8]:
def predict(df, cur_idx):

    regrs = {
        'RF': RandomForestRegressor(min_samples_split=2), 
        'ET': ExtraTreesRegressor(min_samples_split=2), 
        'TF': tfRegr(),
    }
    
    cur = f'case{cur_idx}'

    glm = [f'case{cur_idx - 1}_glm', f'case{cur_idx}_glm'] 
    cols = [f'case{cur_idx - 1}'] + glm
    X, y = df[cols].values, df[cur].values 
    
    
    for name, regr in regrs.items():
        regr = regr.fit(X, y)

        #================ Evaluation ===============START
        y_pred = regr.predict(X).flatten()
        r2 = r2_score(y, y_pred)
        rmse = np.sqrt(((y - y_pred) ** 2).mean())
        print(f'\t{name}: r2 = {r2:.3f}, rmse = {rmse:.3f}')
        #================ Evaluation ===============START
        
        cols = [cur] + glm
        df[cur + f'_{name}_1'] = regr.predict(df[cols].values).flatten()
        
        cols = [cur + f'_{name}_1'] + glm
        df[cur + f'_{name}_2'] = regr.predict(df[cols].values).flatten()
    
    prd_cols_1 = [col for col in df.columns if col.endswith('_1')]
    prd_cols_2 = [col for col in df.columns if col.endswith('_2')]
    prd_1 = df[prd_cols_1].mean(axis=1).values
    prd_2 = df[prd_cols_2].mean(axis=1).values
    
    df.drop(prd_cols_1 + prd_cols_2, axis=1, inplace=True)
    
    return prd_1, prd_2

## Forecast Runs

In [9]:
cur_idx = len(steps) - 1
print(steps[cur_idx])

num_runs = 100
prds_1, prds_2 = [], []

for r in range(num_runs):
    print(f'run = {r}')
    prd_1, prd_2 = predict(df, cur_idx)
    prds_1.append(prd_1)
    prds_2.append(prd_2)

2021-01-16
run = 0
	RF: r2 = 0.939, rmse = 589.922
	ET: r2 = 1.000, rmse = 0.061
	TF: r2 = 0.793, rmse = 1086.418
run = 1
	RF: r2 = 0.966, rmse = 440.597
	ET: r2 = 1.000, rmse = 0.080
	TF: r2 = 0.795, rmse = 1079.732
run = 2
	RF: r2 = 0.969, rmse = 421.336
	ET: r2 = 1.000, rmse = 0.048
	TF: r2 = 0.794, rmse = 1083.399
run = 3
	RF: r2 = 0.966, rmse = 441.110
	ET: r2 = 1.000, rmse = 0.082
	TF: r2 = 0.789, rmse = 1095.614
run = 4
	RF: r2 = 0.960, rmse = 478.730
	ET: r2 = 1.000, rmse = 0.111
	TF: r2 = 0.794, rmse = 1082.749
run = 5
	RF: r2 = 0.976, rmse = 368.247
	ET: r2 = 1.000, rmse = 0.060
	TF: r2 = 0.794, rmse = 1082.065
run = 6
	RF: r2 = 0.972, rmse = 399.452
	ET: r2 = 1.000, rmse = 0.058
	TF: r2 = 0.795, rmse = 1081.669
run = 7
	RF: r2 = 0.961, rmse = 470.810
	ET: r2 = 1.000, rmse = 0.101
	TF: r2 = 0.797, rmse = 1074.109
run = 8
	RF: r2 = 0.945, rmse = 560.406
	ET: r2 = 1.000, rmse = 0.082
	TF: r2 = 0.793, rmse = 1086.518
run = 9
	RF: r2 = 0.936, rmse = 605.785
	ET: r2 = 1.000, rmse 

	RF: r2 = 0.956, rmse = 503.261
	ET: r2 = 1.000, rmse = 0.062
	TF: r2 = 0.793, rmse = 1085.797
run = 80
	RF: r2 = 0.966, rmse = 437.175
	ET: r2 = 1.000, rmse = 0.050
	TF: r2 = 0.794, rmse = 1082.642
run = 81
	RF: r2 = 0.971, rmse = 407.628
	ET: r2 = 1.000, rmse = 0.102
	TF: r2 = 0.792, rmse = 1087.179
run = 82
	RF: r2 = 0.971, rmse = 407.377
	ET: r2 = 1.000, rmse = 0.076
	TF: r2 = 0.786, rmse = 1104.238
run = 83
	RF: r2 = 0.961, rmse = 473.868
	ET: r2 = 1.000, rmse = 0.088
	TF: r2 = 0.789, rmse = 1095.062
run = 84
	RF: r2 = 0.962, rmse = 463.061
	ET: r2 = 1.000, rmse = 0.072
	TF: r2 = 0.789, rmse = 1096.859
run = 85
	RF: r2 = 0.962, rmse = 462.580
	ET: r2 = 1.000, rmse = 0.069
	TF: r2 = 0.792, rmse = 1089.082
run = 86
	RF: r2 = 0.942, rmse = 573.441
	ET: r2 = 1.000, rmse = 0.112
	TF: r2 = 0.793, rmse = 1085.214
run = 87
	RF: r2 = 0.954, rmse = 509.981
	ET: r2 = 1.000, rmse = 0.109
	TF: r2 = 0.794, rmse = 1083.132
run = 88
	RF: r2 = 0.944, rmse = 562.365
	ET: r2 = 1.000, rmse = 0.053
	T

## Prepare for submission

In [10]:
# target end dates
cur_dt = datetime.strptime(steps[cur_idx], '%Y-%m-%d')
fc_dt = (cur_dt + timedelta(days=1)).date()
nxt_dt_1 = (cur_dt + timedelta(days=7)).date()
nxt_dt_2 = (cur_dt + timedelta(days=14)).date()
nxt_dts = [nxt_dt_1, nxt_dt_2]

# quantiles
qs = [.025, .1, .25, .5, .75, .9, .975]

In [11]:
df_county_prd_1 = pd.DataFrame(data=np.array(prds_1).T, index=df.index, columns=range(num_runs))
df_county_prd_2 = pd.DataFrame(data=np.array(prds_2).T, index=df.index, columns=range(num_runs))
county_dfs = [df_county_prd_1, df_county_prd_2]

### County

In [12]:
forecast_dfs = []

for i, county_df in enumerate(county_dfs):
    w = i + 1
    csv_fname = f'results/{steps[cur_idx]}_county_case_{w}-wk_{num_runs}.csv'
    county_df.to_csv(csv_fname, float_format='%.2f')
    
    # ========================= Point esitmate =========================START
    df_point = county_df.mean(axis=1).reset_index()\
        .rename(columns={0: 'value', 'fips': 'location'})
    df_point['type'] = 'point'
    df_point['quantile'] = 'NA'
    # ========================= Point esitmate =========================END

    # ========================= Quantile esitmates =========================START
    df_quantile = county_df.quantile(qs, axis=1).T.reset_index()\
        .melt(id_vars=['fips']).sort_values('fips')\
        .rename(columns={'fips': 'location', 'variable': 'quantile'})
    df_quantile['type'] = 'quantile'
    # ========================= Quantile esitmates =========================END

    df_forecast = pd.concat([df_point, df_quantile])
    df_forecast['target'] = f'{w} wk ahead inc case'
    df_forecast['target_end_date'] = nxt_dts[i]
    
    forecast_dfs.append(df_forecast)

df_forecast_county = pd.concat(forecast_dfs, axis=0).reset_index(drop=True)
print(df_forecast_county.shape)
df_forecast_county.head()

(49504, 6)


location       value   type quantile               target target_end_date
0    01001  285.944307  point       NA  1 wk ahead inc case      2021-01-23
1    01003  792.758095  point       NA  1 wk ahead inc case      2021-01-23
2    01005   79.376841  point       NA  1 wk ahead inc case      2021-01-23
3    01007   78.818818  point       NA  1 wk ahead inc case      2021-01-23
4    01009  181.558328  point       NA  1 wk ahead inc case      2021-01-23

### States

In [13]:
forecast_dfs = []

for i, county_df in enumerate(county_dfs):
    w = i + 1
    
    df_tmp = county_df.reset_index()
    df_tmp['state'] = df_tmp['fips'].apply(lambda x: x[:2])
    state_df = df_tmp.groupby(by='state').sum()
    
    # ========================= Point esitmate =========================START
    df_point = state_df.mean(axis=1).reset_index()\
        .rename(columns={0: 'value', 'state': 'location'})
    df_point['type'] = 'point'
    df_point['quantile'] = 'NA'
    # ========================= Point esitmate =========================END

    # ========================= Quantile esitmates =========================START
    df_quantile = state_df.quantile(qs, axis=1).T.reset_index()\
        .melt(id_vars=['state']).sort_values('state')\
        .rename(columns={'state': 'location', 'variable': 'quantile'})
    df_quantile['type'] = 'quantile'
    # ========================= Quantile esitmates =========================END

    df_forecast = pd.concat([df_point, df_quantile])
    df_forecast['target'] = f'{w} wk ahead inc case'
    df_forecast['target_end_date'] = nxt_dts[i]
    
    forecast_dfs.append(df_forecast)

df_forecast_state = pd.concat(forecast_dfs, axis=0).reset_index(drop=True)
df_forecast_state.head()

location          value   type quantile               target target_end_date
0       01   17256.336453  point       NA  1 wk ahead inc case      2021-01-23
1       02    1655.760522  point       NA  1 wk ahead inc case      2021-01-23
2       04   41987.732870  point       NA  1 wk ahead inc case      2021-01-23
3       05   13604.969829  point       NA  1 wk ahead inc case      2021-01-23
4       06  230037.335077  point       NA  1 wk ahead inc case      2021-01-23

### Nation

In [14]:
forecast_dfs = []

for i, county_df in enumerate(county_dfs):
    w = i + 1
    
    nation_df = pd.DataFrame(data=county_df.sum(axis=0), columns=['US']).T
    nation_df.index.name = 'nation'
    
    # ========================= Point esitmate =========================START
    df_point = nation_df.mean(axis=1).reset_index()\
        .rename(columns={0: 'value', 'nation': 'location'})
    df_point['type'] = 'point'
    df_point['quantile'] = 'NA'
    # ========================= Point esitmate =========================END

    # ========================= Quantile esitmates =========================START
    df_quantile = nation_df.quantile(qs, axis=1).T.reset_index()\
        .melt(id_vars=['nation']).sort_values('nation')\
        .rename(columns={'nation': 'location', 'variable': 'quantile'})
    df_quantile['type'] = 'quantile'
    # ========================= Quantile esitmates =========================END

    df_forecast = pd.concat([df_point, df_quantile])
    df_forecast['target'] = f'{w} wk ahead inc case'
    df_forecast['target_end_date'] = nxt_dts[i]
    
    forecast_dfs.append(df_forecast)

df_forecast_nation = pd.concat(forecast_dfs, axis=0).reset_index(drop=True)
df_forecast_nation

location         value      type quantile               target  \
0        US  1.276030e+06     point       NA  1 wk ahead inc case   
1        US  1.271329e+06  quantile    0.025  1 wk ahead inc case   
2        US  1.273260e+06  quantile      0.1  1 wk ahead inc case   
3        US  1.274882e+06  quantile     0.25  1 wk ahead inc case   
4        US  1.276388e+06  quantile      0.5  1 wk ahead inc case   
5        US  1.277492e+06  quantile     0.75  1 wk ahead inc case   
6        US  1.278415e+06  quantile      0.9  1 wk ahead inc case   
7        US  1.279282e+06  quantile    0.975  1 wk ahead inc case   
8        US  1.185509e+06     point       NA  2 wk ahead inc case   
9        US  1.179964e+06  quantile    0.025  2 wk ahead inc case   
10       US  1.181701e+06  quantile      0.1  2 wk ahead inc case   
11       US  1.183555e+06  quantile     0.25  2 wk ahead inc case   
12       US  1.185595e+06  quantile      0.5  2 wk ahead inc case   
13       US  1.187652e+06  quantile     0.75  2 wk ahead inc case   
14       US  1.189211e+06  quantile      0.9  2 wk ahead inc case   
15       US  1.190921e+06  quantile    0.975  2 wk ahead inc case   

   target_end_date  
0       2021-01-23  
1       2021-01-23  
2       2021-01-23  
3       2021-01-23  
4       2021-01-23  
5       2021-01-23  
6       2021-01-23  
7       2021-01-23  
8       2021-01-30  
9       2021-01-30  
10      2021-01-30  
11      2021-01-30  
12      2021-01-30  
13      2021-01-30  
14      2021-01-30  
15      2021-01-30

In [15]:
df_forecast = pd.concat([df_forecast_county, df_forecast_state, df_forecast_nation])
df_forecast.reset_index(drop=True, inplace=True)
df_forecast['forecast_date'] = fc_dt
cols = ['forecast_date', 'target', 'target_end_date', 'location', 'type', 'quantile', 'value']
df_forecast = df_forecast[cols]

# =============== make nagative value zero ==============
value_dezero = df_forecast['value']
value_dezero[value_dezero < 0]  = 0
df_forecast['value'] = value_dezero

In [16]:
team = 'UChicagoCHATTOPADHYAY'
model = 'UnIT'
df_forecast.to_csv(f'results/{fc_dt}-{team}-{model}_case.csv', float_format='%.2f', index=False)

In [17]:
validation = '/home/yihuang/Documents/Data/covid19-forecast-hub/code/validation/validate_single_forecast_file.py'

! python3 {validation} results/2020-11-08-UChicagoCHATTOPADHYAY-UnIT_case.csv

/Library/Frameworks/Python.framework/Versions/3.9/bin/python3: can't open file '/home/yihuang/Documents/Data/covid19-forecast-hub/code/validation/validate_single_forecast_file.py': [Errno 2] No such file or directory
